### ℹ️ **Info**
* **forked original great work kernels**

    * https://www.kaggle.com/code/ambrosm/adc24-intro-inference
    * https://www.kaggle.com/code/hugowjd/neurips-ariel-2024-starter
    * https://www.kaggle.com/code/xiaocao123/neurips-ariel-2024-starter?scriptVersionId=193156800
    * https://www.kaggle.com/code/bingyuniu/neurips-ariel-2024-starter-withdifferentparametr

* **2024/08/26 My Additional**
    * percentile FE add.
```
np.percentile(window, 5, axis=1),
np.percentile(window, 10, axis=1),
np.percentile(window, 15, axis=1),
np.percentile(window, 20, axis=1),
np.percentile(window, 25, axis=1),
np.percentile(window, 30, axis=1),
np.percentile(window, 35, axis=1),
np.percentile(window, 40, axis=1),
np.percentile(window, 60, axis=1),
np.percentile(window, 65, axis=1),
np.percentile(window, 70, axis=1),
np.percentile(window, 75, axis=1),
np.percentile(window, 80, axis=1),
np.percentile(window, 85, axis=1),
np.percentile(window, 90, axis=1),
np.percentile(window, 95, axis=1),
np.median(window, axis=1),
np.var(window, axis=1),
```

* **2024/08/29 My Additional**
    * add FE
```
f_sliding_features2 = sliding_window_features(f_raw, 100, 10)
a_sliding_features2 = sliding_window_features(a_raw, 100, 10)
```



In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import polars as pl
import numpy as np
import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import pickle
import time
import os
import pickle
import seaborn as sns
import scipy.stats
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import kurtosis
from scipy.stats import skew

# Pre-Processing
## Load Functions

In [2]:
%%writefile utils.py
import pandas as pd
import polars as pl
import numpy as np
from tqdm import tqdm
import pickle
PATH = "/kaggle/input/ariel-data-challenge-2024"
def load_signal_data(planet_id, dataset, instrument, img_size):
    file_path = f'{PATH}/{dataset}/{planet_id}/{instrument}_signal.parquet'
    signal = pl.read_parquet(file_path)
    mean_signal = signal.cast(pl.Int32).sum_horizontal().cast(pl.Float32).to_numpy() / img_size # mean over the 32*32 pixels
    net_signal = mean_signal[1::2] - mean_signal[0::2]
    return net_signal

def read_and_preprocess(dataset, planet_ids, instrument = "AIRS-CH0"):
    """Read the files for all planet_ids and extract the time series.
    Parameters
    dataset: 'train' or 'test'
    planet_ids: list of planet ids
    instrument: the instrument of observation, 'AIRS-CH0' or 'FGS1', default to 'AIRS-CH0'
    Returns
    dataframe with one row per planet_id and 67500 values per row for FGS1 and 5624 for AIRS-CH0
    """
    img_size = 1024 if instrument == "FGS1" else 32*356
    column_num = 67500 if instrument == 'FGS1' else 5625
    raw_train = np.full((len(planet_ids), column_num), np.nan, dtype=np.float32)
    for i, planet_id in tqdm(list(enumerate(planet_ids))):
        raw_train[i] = load_signal_data(planet_id, dataset, instrument, img_size)
    return raw_train

def feature_engineering(f_raw, a_raw, adc_info, window_size=60, step_size=15):
    """Create a dataframe with combined features from the raw data, including sliding window and time-series statistics.
    
    Parameters:
    f_raw: ndarray of shape (n_planets, 67500)
    a_raw: ndarray of shape (n_planets, 5625)
    window_size: int, size of the sliding window for time-series statistics
    step_size: int, step size for the sliding window
    
    Return value:
    df: DataFrame of shape (n_planets, several features)
    """
    f_obscured = f_raw[:, 23500:44000].mean(axis=1)
    f_unobscured = (f_raw[:, :20500].mean(axis=1) + f_raw[:, 47000:].mean(axis=1)) / 2
    f_relative_reduction = (f_unobscured - f_obscured) / f_unobscured
    f_std_dev = f_raw.std(axis=1)
    f_signal_to_noise = f_unobscured / f_std_dev

    a_obscured = a_raw[:, 1958:3666].mean(axis=1)
    a_unobscured = (a_raw[:, :1708].mean(axis=1) + a_raw[:, 3916:].mean(axis=1)) / 2
    a_relative_reduction = (a_unobscured - a_obscured) / a_unobscured
    a_std_dev = a_raw.std(axis=1)
    a_signal_to_noise = a_unobscured / a_std_dev

    f_variance = f_raw.var(axis=1)
    a_variance = a_raw.var(axis=1)
    
    f_skewness = pd.DataFrame(f_raw).skew(axis=1).values
    a_skewness = pd.DataFrame(a_raw).skew(axis=1).values

    f_kurtosis = pd.DataFrame(f_raw).kurtosis(axis=1).values
    a_kurtosis = pd.DataFrame(a_raw).kurtosis(axis=1).values
    
    f_half_obscured1 = f_raw[:, 20500:23500].mean(axis=1)
    f_half_obscured2 = f_raw[:, 44000:47000].mean(axis=1)
    f_half_reduction1 = (f_unobscured - f_half_obscured1) / f_unobscured
    f_half_reduction2 = (f_unobscured - f_half_obscured2) / f_unobscured

    a_half_obscured1 = a_raw[:, 1708:1958].mean(axis=1)
    a_half_obscured2 = a_raw[:, 3666:3916].mean(axis=1)
    a_half_reduction1 = (a_unobscured - a_half_obscured1) / a_unobscured
    a_half_reduction2 = (a_unobscured - a_half_obscured2) / a_unobscured

    # Sliding window features
    def sliding_window_features(data, window_size, step_size):
        features = []
        max_index = data.shape[1]
        for start in range(0, max_index - window_size + 1, step_size):
            end = start + window_size
            window = data[:, start:end]
            features.append([
                np.mean(window, axis=1),
                np.std(window, axis=1),
                np.min(window, axis=1),
                np.max(window, axis=1),
                np.percentile(window, 5, axis=1),
                np.percentile(window, 10, axis=1),
                np.percentile(window, 15, axis=1),
                np.percentile(window, 20, axis=1),
                np.percentile(window, 25, axis=1),
                np.percentile(window, 30, axis=1),
                np.percentile(window, 35, axis=1),
                np.percentile(window, 40, axis=1),
                np.percentile(window, 60, axis=1),
                np.percentile(window, 65, axis=1),
                np.percentile(window, 70, axis=1),
                np.percentile(window, 75, axis=1),
                np.percentile(window, 80, axis=1),
                np.percentile(window, 85, axis=1),
                np.percentile(window, 90, axis=1),
                np.percentile(window, 95, axis=1),
                np.median(window, axis=1),
                np.var(window, axis=1),
#                 kurtosis(window, axis=1),
#                 skew(window, axis=1),
            ])
        if features:
            return np.vstack(features).T  # Stack vertically and transpose to get the correct shape
        else:
            return np.empty((data.shape[0], 0))  # Return empty array with correct shape
    
    f_sliding_features = sliding_window_features(f_raw, window_size, step_size)
    a_sliding_features = sliding_window_features(a_raw, window_size, step_size)
    
    f_sliding_features2 = sliding_window_features(f_raw, 150, 10)
    a_sliding_features2 = sliding_window_features(a_raw, 150, 10)


    print(f'f_sliding_features.shape: {f_sliding_features.shape}')
    print(f'a_sliding_features.shape: {a_sliding_features.shape}')


    df = pd.DataFrame({
        'f_relative_reduction': f_relative_reduction,
        'f_signal_to_noise': f_signal_to_noise,
        'f_variance': f_variance,
        'f_skewness': f_skewness,
        'f_kurtosis': f_kurtosis,
        'a_relative_reduction': a_relative_reduction,
        'a_signal_to_noise': a_signal_to_noise,
        'a_variance': a_variance,
        'a_skewness': a_skewness,
        'a_kurtosis': a_kurtosis,
        'f_half_reduction1': f_half_reduction1,
        'f_half_reduction2': f_half_reduction2,
        'a_half_reduction1': a_half_reduction1,
        'a_half_reduction2': a_half_reduction2
    })


    if f_sliding_features.size > 0:
        f_sliding_df = pd.DataFrame(f_sliding_features, columns=[f'f_slide_{i}' for i in range(f_sliding_features.shape[1])])
        df = pd.concat([df, f_sliding_df], axis=1)
    if a_sliding_features.size > 0:
        a_sliding_df = pd.DataFrame(a_sliding_features, columns=[f'a_slide_{i}' for i in range(a_sliding_features.shape[1])])
        df = pd.concat([df, a_sliding_df], axis=1)
    
    if f_sliding_features2.size > 0:
        f_sliding_df = pd.DataFrame(f_sliding_features2, columns=[f'f_slide2_{i}' for i in range(f_sliding_features2.shape[1])])
        df = pd.concat([df, f_sliding_df], axis=1)
    if a_sliding_features2.size > 0:
        a_sliding_df = pd.DataFrame(a_sliding_features2, columns=[f'a_slide2_{i}' for i in range(a_sliding_features2.shape[1])])
        df = pd.concat([df, a_sliding_df], axis=1)
    
    
    df = pd.concat([df, adc_info.reset_index().iloc[:, 1:6]], axis=1)
    
    return df

Writing utils.py


In [3]:
# Load Meta-Data
PATH = "/kaggle/input/ariel-data-challenge-2024"
train_adc_info = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/train_adc_info.csv', 
                             index_col='planet_id')
train_labels = pd.read_csv('/kaggle/input/ariel-data-challenge-2024/train_labels.csv',
                           index_col='planet_id')
wavelengths = pd.read_csv(f'{PATH}/wavelengths.csv')

In [4]:
exec(open('utils.py', 'r').read())

In [5]:
%%writefile -a utils.py

def postprocessing(pred_array, index, sigma_pred):
    """Create a submission dataframe from its components
    
    Parameters:
    pred_array: ndarray of shape (n_samples, 283)
    index: pandas.Index of length n_samples with name 'planet_id'
    sigma_pred: float
    
    Return value:
    df: DataFrame of shape (n_samples, 566) with planet_id as index
    """
    return pd.concat([pd.DataFrame(pred_array.clip(0, None), index=index, columns=wavelengths.columns),
                      pd.DataFrame(sigma_pred, index=index, columns=[f"sigma_{i}" for i in range(1, 284)])],
                     axis=1)

class ParticipantVisibleError(Exception):
    pass

def competition_score(
        solution: pd.DataFrame,
        submission: pd.DataFrame,
        naive_mean: float,
        naive_sigma: float,
        sigma_true: float,
        row_id_column_name='planet_id',
    ) -> float:
    '''
    This is a Gaussian Log Likelihood based metric. For a submission, which contains the predicted mean (x_hat) and variance (x_hat_std),
    we calculate the Gaussian Log-likelihood (GLL) value to the provided ground truth (x). We treat each pair of x_hat,
    x_hat_std as a 1D gaussian, meaning there will be 283 1D gaussian distributions, hence 283 values for each test spectrum,
    the GLL value for one spectrum is the sum of all of them.

    Inputs:
        - solution: Ground Truth spectra (from test set)
            - shape: (nsamples, n_wavelengths)
        - submission: Predicted spectra and errors (from participants)
            - shape: (nsamples, n_wavelengths*2)
        naive_mean: (float) mean from the train set.
        naive_sigma: (float) standard deviation from the train set.
        sigma_true: (float) essentially sets the scale of the outputs.
    '''

    del solution[row_id_column_name]
    del submission[row_id_column_name]

    if submission.min().min() < 0:
        raise ParticipantVisibleError('Negative values in the submission')
    for col in submission.columns:
        if not pd.api.types.is_numeric_dtype(submission[col]):
            raise ParticipantVisibleError(f'Submission column {col} must be a number')

    n_wavelengths = len(solution.columns)
    if len(submission.columns) != n_wavelengths*2:
        raise ParticipantVisibleError('Wrong number of columns in the submission')

    y_pred = submission.iloc[:, :n_wavelengths].values
    # Set a non-zero minimum sigma pred to prevent division by zero errors.
    sigma_pred = np.clip(submission.iloc[:, n_wavelengths:].values, a_min=10**-15, a_max=None)
    y_true = solution.values

    GLL_pred = np.sum(scipy.stats.norm.logpdf(y_true, loc=y_pred, scale=sigma_pred))
    GLL_true = np.sum(scipy.stats.norm.logpdf(y_true, loc=y_true, scale=sigma_true * np.ones_like(y_true)))
    GLL_mean = np.sum(scipy.stats.norm.logpdf(y_true, loc=naive_mean * np.ones_like(y_true), scale=naive_sigma * np.ones_like(y_true)))

    submit_score = (GLL_pred - GLL_mean)/(GLL_true - GLL_mean)
    return float(np.clip(submit_score, 0.0, 1.0))

Appending to utils.py


## Load Data

In [6]:
%%time
if os.path.exists("/kaggle/input/adc24-intro-training/f_raw_train.pickle"):
    f_raw_train = np.load('/kaggle/input/adc24-intro-training/f_raw_train.pickle', allow_pickle=True)
else:
    f_raw_train = read_and_preprocess('train', train_labels.index, 'FGS1')
    with open('f_raw_train.pickle', 'wb') as f:
        pickle.dump(f_raw_train, f)

CPU times: user 0 ns, sys: 204 ms, total: 204 ms
Wall time: 1.87 s


In [7]:
%%time
if os.path.exists("/kaggle/input/adc24-intro-training/a_raw_train.pickle"):
    a_raw_train = np.load('/kaggle/input/adc24-intro-training/a_raw_train.pickle', allow_pickle=True)
else:
    a_raw_train = read_and_preprocess('train', train_labels.index)
    with open('a_raw_train.pickle', 'wb') as f:
        pickle.dump(a_raw_train, f)

CPU times: user 5.74 ms, sys: 13.6 ms, total: 19.4 ms
Wall time: 106 ms


## Feature Engineering

In [8]:
%%time
train = feature_engineering(f_raw_train, a_raw_train, train_adc_info)


f_sliding_features.shape: (673, 98934)
a_sliding_features.shape: (673, 8184)
CPU times: user 5min 49s, sys: 6.32 s, total: 5min 55s
Wall time: 5min 56s


In [9]:
train.head()
print(train.shape)


(673, 267385)


In [10]:
train_labels.head()

print(train_labels.shape)

(673, 283)


In [11]:
train = train.iloc[:,:-1]

In [14]:
# 划分训练集和测试集
train_data = train.iloc[:471]   # 前 471 行作为训练集
test_data = train.iloc[471:]    # 后 202 行作为测试集

train_labels_split = train_labels.iloc[:471]  # 对应的训练标签
test_labels_split = train_labels.iloc[471:]  # 对应的测试标签

# 打印数据形状，确保划分正确
print(f"train_data.shape: {train_data.shape}")   # 应为 (471, 267384)
print(f"test_data.shape: {test_data.shape}")     # 应为 (202, 267384)
print(f"train_labels_split.shape: {train_labels_split.shape}")  # 应为 (471,)
print(f"test_labels_split.shape: {test_labels_split.shape}")    # 应为 (202,)


train_data.shape: (471, 267384)
test_data.shape: (202, 267384)
train_labels_split.shape: (471, 283)
test_labels_split.shape: (202, 283)


In [15]:
# PCA
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# 假设 train 是一个 pandas DataFrame，每行代表一个行星，每列代表一个特征

# 检查缺失值并填补
if train_data.isnull().values.any():
    train_data = train_data.fillna(train_data.mean())
if test_data.isnull().values.any():
    test_data = test_data.fillna(test_data.mean())
    
# 分离特征
X_train = train_data.values  # 或者 train_data.to_numpy()
X_test = test_data.values    # 或者 test_data.to_numpy()

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# 初始化 PCA，保留 200 个主成分
pca = PCA(n_components=200, random_state=42)

# 拟合 PCA 并转换训练数据
X_train_pca = pca.fit_transform(X_train_scaled)

# 创建训练集主成分的列名
pca_columns = [f'PC{i}' for i in range(1, X_train_pca.shape[1] + 1)]

# 转换为 DataFrame
train_pca_df = pd.DataFrame(X_train_pca, index=train_data.index, columns=pca_columns)

# 对测试集进行标准化，并转换到相同的 PCA 空间
X_test_scaled = scaler.transform(X_test)  # 使用训练集的标准化参数

# 确保测试集是二维数据
print(f"X_test_scaled shape: {X_test_scaled.shape}")  # 应为 (202, 267384)

X_test_pca = pca.transform(X_test_scaled)

# 创建测试集主成分的列名
test_pca_columns = [f'PC{i}' for i in range(1, X_test_pca.shape[1] + 1)]

# 转换为 DataFrame
test_pca_df = pd.DataFrame(X_test_pca, index=test_data.index, columns=test_pca_columns)

# 查看降维后的训练集和测试集数据形状
print(f"训练集 PCA 结果形状: {train_pca_df.shape}")
print(f"测试集 PCA 结果形状: {test_pca_df.shape}")


X_test_scaled shape: (202, 267384)
训练集 PCA 结果形状: (471, 200)
测试集 PCA 结果形状: (202, 200)


In [16]:
# 保存Scaler和PCA模型
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open('pca_model.pkl', 'wb') as f:
    pickle.dump(pca, f)


# Model
## Xgboost Model

In [17]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# print(train.shape)
# print(train)
    
model = XGBRegressor(
    n_estimators=50,
    learning_rate=0.01,
    max_depth=3,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
        n_jobs=-1
)

# 训练模型
model.fit(train_pca_df, train_labels_split)

# 对测试集进行预测
test_pred = model.predict(test_pca_df)
print(test_pred.shape)

# 测试集评估：计算 R2 和 RMSE
print(f"# R2 score on test set: {r2_score(test_labels_split, test_pred):.4f}")
sigma_pred_test = mean_squared_error(test_labels_split, test_pred, squared=False)
print(f"# Root Mean Squared Error on test set: {sigma_pred_test:.7f}")



(202, 283)
# R2 score on test set: 0.4686
# Root Mean Squared Error on test set: 0.0012806


In [22]:
# GLL
import numpy as np
import math

def calculate_gll(y_pred=0,y_true=0,sigmas=1e-5):
    var = sigmas**2
    mse = (y_pred - y_true)**2
    score = -0.5*(np.log(2*math.pi)+np.log(var)+mse/var)
    return round(score.mean(),3)


# L_ref
mn = np.mean(test_labels_split.values, axis=0, keepdims=True)
stdev = np.std(test_labels_split.values, axis=0, keepdims=True)
L_ref = calculate_gll(mn,test_labels_split,stdev)
L_ref_mean = round(L_ref.mean(), 3)
print("L_ref_mean = ",L_ref_mean)

# L_ideal
L_ideal = calculate_gll()
L_ideal_mean = round(L_ideal.mean(),3)
print("L_ideal = ",L_ideal_mean)

# L
mn_test = np.mean(test_pred, axis=0, keepdims=True)
stdev_test = np.std(test_pred, axis=0, keepdims=True)
L = calculate_gll(mn_test, test_pred, stdev_test)
L_mean = round(L.mean(), 3)
print("L = ",L_mean)


L_ref_mean =  4.925
L_ideal =  10.594
L =  6.089


In [23]:
# Score
score = (L_mean - L_ref_mean)/(L_ideal_mean - L_ref_mean)
print("Score = ",score)

Score =  0.2053272579056516


In [ ]:
test_pred_df = pd.DataFrame(test_pred, columns=[f"Feature_{i+1}" for i in range(test_pred.shape[1])])
test_pred_df.columns = test_labels_split.columns
# test_pred_df.index = test_labels_split.index


print("True")
print(test_labels_split.head())

print("Predict")
print(test_pred_df.head())



In [ ]:
import matplotlib.pyplot as plt

# 提取前10个星球的真实值和预测值
test_labels_split_top10 = test_labels_split.iloc[:10, :]
test_pred_df_top10 = test_pred_df.iloc[:10, :]

# 设置画布大小
fig, axes = plt.subplots(5, 2, figsize=(15, 15))  # 5行2列，共十张图

# 遍历前10个星球
for i in range(10):
    # 选择第i个星球的真实值和预测值
    true_values = test_labels_split_top10.iloc[i, :]
    predicted_values = test_pred_df_top10.iloc[i, :]

    # 计算当前子图位置
    ax = axes[i // 2, i % 2]

    # 画出真实值和预测值
    ax.plot(range(283), true_values, label="True Values", color='blue', linestyle='-')
    ax.plot(range(283), predicted_values, label="Predicted Values", color='red', linestyle='-')

    # 设置标题和标签
    ax.set_title(f"Planet {i + 1}")
    ax.set_xlabel('Feature')
    ax.set_ylabel('Value')

    ax.set_ylim(0, 0.05)
    
    ax.legend()

# 调整子图之间的间距
plt.tight_layout()

# 显示图表
plt.show()



## Inference